In [154]:
import numpy as np

In [346]:
class RSA():
    def __init__(self):
        self.bits = 16
        self.block_size = 4
        
    def primes_in_range(self, x, y):
        for n in range(x, y):
            isPrime = True

            for num in range(2, n):
                if n % num == 0:
                    isPrime = False

            if isPrime:
                return n

        return None
    
    def random_prime(self, bits):
        prime_list = self.primes_in_range(2**bits, 2**bits+1000)
        return np.random.choice(prime_list)
    
    def generate_keys(self):      
#         p = self.random_prime(self.bits)
#         q = self.random_prime(self.bits)
#         e = self.random_prime(self.bits)
        p = 1299359  
        q = 1297421
        e = 173
        n = p*q
        phi = (p-1)*(q-1)
        d = self.modinv(e, phi)
        
        private_key = {'mod': n, 'key': d}
        public_key = {'mod': n, 'key': e}
        return private_key, public_key

    def encrypt(self, plaintext, public_key):
        blocks = [plaintext[i:i+self.block_size] for i in range(0, len(plaintext), self.block_size)]
        diff = self.block_size - len(blocks[-1]) 
        blocks[-1] = blocks[-1] + ' '*diff
        encrypted_blocks = []
        e = int(public_key["key"])
        n = int(public_key["mod"])

        for block in blocks:
            m = int.from_bytes(bytearray(block.encode()), byteorder='big', signed=False) 
            c = pow(m, e, n)
            encrypted_blocks.append(str(c))       
        return ' '.join(encrypted_blocks)
    
    def decrypt(self, ciphertext, private_key):
        encrypted_blocks = ciphertext.split(' ')
        res = ''
        d = int(private_key["key"])
        n = int(private_key["mod"])
        
        for block in encrypted_blocks:  
            m = pow(int(block), d, n)
            res += int.to_bytes(m, length=self.block_size, byteorder='big', signed=False).decode()  
            
        return res
    
    def egcd(self, n: int, m: int):
        """
        Extended Euclidean Algorithm 
        """
        if m == 0:
            return n, 0, 1

        gcd, x, y = self.egcd(m, n % m)

        x1 = y - (n//m) * x
        y1 = x
        return gcd, x1, y1

    def modinv(self, a: int, m: int): 
        """
        D(x)=a^-1 * (x - b) mod m
        a: key
        m: mod %
        """
        gcd, x, y = self.egcd(a, m) 
        if gcd != 1:
            return None
        else: 
            return y % m

In [347]:
rsa = RSA()

In [348]:
private_key, public_key = rsa.generate_keys()

In [349]:
private_key

{'mod': 1685815653139, 'key': 204636266957}

In [350]:
ciphertext = rsa.encrypt("helolsc sdc", public_key)

In [351]:
rsa.decrypt(ciphertext, private_key)

'helolsc sdc '